In [1]:
# server_ip = "cloudflare-quic.com/"
server_ip = "3.73.33.119"
iperf_port = "6969"

## Bandwidth and latency test

### Latency

In [2]:
!ping -c 5 {server_ip}

PING 3.73.33.119 (3.73.33.119) 56(84) bytes of data.
64 bytes from 3.73.33.119: icmp_seq=1 ttl=63 time=0.432 ms
64 bytes from 3.73.33.119: icmp_seq=2 ttl=63 time=0.391 ms
64 bytes from 3.73.33.119: icmp_seq=3 ttl=63 time=0.376 ms
64 bytes from 3.73.33.119: icmp_seq=4 ttl=63 time=0.423 ms
64 bytes from 3.73.33.119: icmp_seq=5 ttl=63 time=0.387 ms

--- 3.73.33.119 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4093ms
rtt min/avg/max/mdev = 0.376/0.401/0.432/0.021 ms


### Bandwidth - TCP

In [3]:
!iperf -c {server_ip} -p {iperf_port}

Connecting to host 3.73.33.119, port 6969
[  5] local 172.31.7.227 port 44306 connected to 3.73.33.119 port 6969
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec   377 MBytes  3.16 Gbits/sec  620    365 KBytes       
[  5]   1.00-2.00   sec   536 MBytes  4.50 Gbits/sec  1503    358 KBytes       
[  5]   2.00-3.00   sec   554 MBytes  4.64 Gbits/sec  1805    440 KBytes       
[  5]   3.00-4.00   sec   506 MBytes  4.25 Gbits/sec  781    424 KBytes       
[  5]   4.00-5.00   sec   564 MBytes  4.73 Gbits/sec  1181    378 KBytes       
[  5]   5.00-6.00   sec   562 MBytes  4.72 Gbits/sec  435    332 KBytes       
[  5]   6.00-7.00   sec   562 MBytes  4.72 Gbits/sec  1605    280 KBytes       
[  5]   7.00-8.00   sec   564 MBytes  4.73 Gbits/sec  585    744 KBytes       
[  5]   8.00-9.00   sec   561 MBytes  4.70 Gbits/sec  350    368 KBytes       
[  5]   9.00-10.00  sec   562 MBytes  4.73 Gbits/sec  1605    389 KBytes       
- - - - - - - - - - - - - -

### Bandwidth - UDP

In [4]:
!iperf -u -c {server_ip} -p {iperf_port} -b 0

Connecting to host 3.73.33.119, port 6969
[  5] local 172.31.7.227 port 56190 connected to 3.73.33.119 port 6969
[ ID] Interval           Transfer     Bitrate         Total Datagrams
[  5]   0.00-1.00   sec   572 MBytes  4.80 Gbits/sec  414370  
[  5]   1.00-2.00   sec   573 MBytes  4.81 Gbits/sec  414970  
[  5]   2.00-3.00   sec   574 MBytes  4.81 Gbits/sec  415470  
[  5]   3.00-4.00   sec   574 MBytes  4.81 Gbits/sec  415390  
[  5]   4.00-5.00   sec   574 MBytes  4.81 Gbits/sec  415470  
[  5]   5.00-6.00   sec   574 MBytes  4.81 Gbits/sec  415410  
[  5]   6.00-7.00   sec   573 MBytes  4.80 Gbits/sec  414740  
[  5]   7.00-8.00   sec   574 MBytes  4.82 Gbits/sec  415880  
[  5]   8.00-9.00   sec   574 MBytes  4.81 Gbits/sec  415440  
[  5]   9.00-10.00  sec   574 MBytes  4.81 Gbits/sec  415560  
- - - - - - - - - - - - - - - - - - - - - - - - -
[ ID] Interval           Transfer     Bitrate         Jitter    Lost/Total Datagrams
[  5]   0.00-10.00  sec  5.60 GBytes  4.81 Gbits/sec

### Measuring QUIC performance

#### h2 load output parsing

In [5]:
def get_throughput(res):
    return float(res[8].split(" ")[-1][:-4])

def get_rps(res):
    return float(res[17].split(" ")[3])

#### Bulk Throughput

Get throughput measurment from the client.

In [10]:
  # --no-udp-gso
  #             Disable UDP GSO.
  # --max-udp-payload-size=<SIZE>
  #             Specify the maximum outgoing UDP datagram payload size.


for protocol in ("h3",):
    print(f"{protocol} - bulk throughput")
    # Request the file first, just to warm up caches.
    _ = !h2load -n1 -c1 --npn-list={protocol} https://{server_ip}/2g.txt
    for i in range(1):
        res = !h2load -n1 -c1 --npn-list={protocol} https://{server_ip}/2g.txt --qlog-file-base="throughput"
        t = get_throughput(res)
        print(f"{t} MB/s")

h3 - bulk throughput
88.7 MB/s


In [8]:
for protocol in ("h2", "h3"):
    print(f"{protocol} - rps test")
    for i in range(5):
        res = !h2load -n1000 -c1 --npn-list={protocol} https://{server_ip}
        rps = get_rps(res)
        print(f"{rps} req/s")

h2 - rps test
2381.97 req/s
2519.08 req/s
2437.94 req/s
2516.84 req/s
2438.93 req/s
h3 - rps test
38.99 req/s
39.01 req/s
38.93 req/s
38.98 req/s
38.98 req/s
